In [29]:
import cvxpy as cp
import numpy as np
from numpy.typing import NDArray

def get_ordered_comparison_model(line_array : NDArray, epsilon=1e-5, M = 1e5):
    ordered_list = get_list_of_arrays_less_than_k(line_array)
    Cis = cp.Variable(len(line_array), boolean=True)
    constraints_hi = []
    for idx in range(len(ordered_list)):
        constraints_hi += [ordered_list[idx][idx] >= ordered_list[k] + epsilon - M*(1-Cis[idx])
                          for k in range(len(ordered_list[idx])-1)]
    return Cis, constraints_hi

def get_list_of_arrays_less_than_k(line_array : NDArray):
    return [line_array[:k+1] for k in range(len(line_array))]

# Example data
A = np.array([10, 15, 12, 20,10, 40,41, 52, 53, 23, 22, 22]) 
print(get_list_of_arrays_less_than_k(A))
epsilon = 1e-5  # Tolerance for strict inequality
M = 1e5  # Big-M value

Cis, constraints = get_ordered_comparison_model(A, epsilon, M)
objective = cp.Maximize(cp.sum(Cis))
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

print(problem.status)

# Results
print(f"Number of towers to see: {problem.value}")

[array([10]), array([10, 15]), array([10, 15, 12]), array([10, 15, 12, 20]), array([10, 15, 12, 20, 10]), array([10, 15, 12, 20, 10, 40]), array([10, 15, 12, 20, 10, 40, 41]), array([10, 15, 12, 20, 10, 40, 41, 52]), array([10, 15, 12, 20, 10, 40, 41, 52, 53]), array([10, 15, 12, 20, 10, 40, 41, 52, 53, 23]), array([10, 15, 12, 20, 10, 40, 41, 52, 53, 23, 22]), array([10, 15, 12, 20, 10, 40, 41, 52, 53, 23, 22, 22])]
optimal
Number of towers to see: 7.0
